# 3-D Density Kernel Estimation for Counting in Microscopy Image Volumes Using 3-D Image Filters and Random Decision Trees

#### By Dominic Waithe 

From journal paper:  
Waithe, Dominic, et al. "3-D Density Kernel Estimation for Counting in Microscopy Image Volumes Using 3-D Image Filters and Random Decision Trees." European Conference on Computer Vision. Springer International Publishing, 2016.

## The Functions and dependencies used for the 3D Density Estimation

In [ ]:
%pylab inline
#Dependencies.
import vigra
import sys
from sklearn.cross_validation import ShuffleSplit
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from scipy.ndimage import measurements, center_of_mass, filters
from scipy.ndimage import gaussian_gradient_magnitude, gaussian_laplace, gaussian_filter
from sklearn.ensemble import ExtraTreesRegressor
import time
import tifffile as tif_fn


def evaluate_forest_3D(par_obj,model_num,count=None):
    """Evaluates the forest at a particular slice"""
    mimg_lin = np.reshape(par_obj.feat_arr[count], \
                          (par_obj.height * par_obj.width, par_obj.feat_arr[count].shape[2]))
    linPred = par_obj.RF[model_num].predict(mimg_lin)/4.0
    par_obj.pred_arr[count] = linPred.reshape(par_obj.height, par_obj.width)
    maxPred = np.max(linPred)
    sum_pred =np.sum(linPred/255)
    par_obj.sum_pred[count] = sum_pred

def local_shape_features_2D(im,scaleStart):
    """2-D Image Shape features"""
    s = scaleStart
    
    imSizeC = im.shape[0]
    imSizeR = im.shape[1]
    f = np.zeros((imSizeC,imSizeR,21))
    
    st08 = vigra.filters.structureTensorEigenvalues(im,s*1,s*2)
    st16 = vigra.filters.structureTensorEigenvalues(im,s*2,s*4)
    st32 = vigra.filters.structureTensorEigenvalues(im,s*4,s*8)
    st64 = vigra.filters.structureTensorEigenvalues(im,s*8,s*16)
    st128 = vigra.filters.structureTensorEigenvalues(im,s*16,s*32)
    
    f[:,:, 0]  =  im
    f[:,:, 1]  =  gaussian_gradient_magnitude(im, s)
    f[:,:, 2]  =  st08[:,:,0]
    f[:,:, 3]  =  st08[:,:,1]
    f[:,:, 4]  =  gaussian_laplace(im, s )
    f[:,:, 5]  =  gaussian_gradient_magnitude(im, s*2) 
    f[:,:, 6]  =  st16[:,:,0]
    f[:,:, 7]  =  st16[:,:,1]
    f[:,:, 8]  =  gaussian_laplace(im, s*2 )
    f[:,:, 9]  =  gaussian_gradient_magnitude(im, s*4) 
    f[:,:, 10] =  st32[:,:,0]
    f[:,:, 11] =  st32[:,:,1]
    f[:,:, 12] =  gaussian_laplace(im, s*4 )
    f[:,:, 13]  = gaussian_gradient_magnitude(im, s*8) 
    f[:,:, 14] =  st64[:,:,0]
    f[:,:, 15] =  st64[:,:,1]
    f[:,:, 16] =  gaussian_laplace(im, s*8 )
    f[:,:, 17]  = gaussian_gradient_magnitude(im, s*16) 
    f[:,:, 18] =  st128[:,:,0]
    f[:,:, 19] =  st128[:,:,1]
    f[:,:, 20] =  gaussian_laplace(im, s*16 )
    return f
def local_shape_features_3D(input_s,scaleStart):
    """3-D Image Shape features"""
    s = scaleStart
    
    imSizeZ = input_s.shape[0]
    imSizeC = input_s.shape[1]
    imSizeR = input_s.shape[2]
    
    
    #If the input is small we need to zero-pad for the Eigenvalue determination.
    if imSizeZ < 40:    
        input_ss =  np.zeros((40,imSizeC,imSizeR))
        input_ss[0:imSizeZ,:,:] = input_s
    else:
        input_ss = input_s
        
    f = np.zeros((imSizeZ,imSizeC,imSizeR,25))
    
    f[:,:,:,0] = np.array(input_s)
    
    f[:,:,:,1] = gaussian_gradient_magnitude(input_s, [(s/1),s,s])
    f[:,:,:,2] = gaussian_laplace(input_s,[(s/1),s,s])
    f[:,:,:,3] =  gaussian_filter(input_s,[(s/1),s,s])
    fe = vigra.filters.structureTensorEigenvalues(input_ss.astype(np.float32),\
                                                  [(s/1),s,s],[(s/1)*2,s*2,s*2])
    f[:,:,:,4] = fe[:,:,:,0]
    f[:,:,:,5] = fe[:,:,:,1]
    f[:,:,:,6] = fe[:,:,:,2]  
    
    f[:,:,:,7] = gaussian_gradient_magnitude(input_s, [(s/1)*2,s*2,s*2])
    f[:,:,:,8] = gaussian_laplace(input_s,[(s/1)*2,s*2,s*2])
    f[:,:,:,9] =  gaussian_filter(input_s,[(s/1)*2,s*2,s*2])
    fe = vigra.filters.structureTensorEigenvalues(input_ss.astype(np.float32),\
                                                  [(s/1)*2,s*2,s*2],[(s/1)*4,s*4,s*4])
    f[:,:,:,10] = fe[:,:,:,0]
    f[:,:,:,11] = fe[:,:,:,1]
    f[:,:,:,12] = fe[:,:,:,2]    
    
    f[:,:,:,13] = gaussian_gradient_magnitude(input_s, [(s/1)*4,s*4,s*4])
    f[:,:,:,14] = gaussian_laplace(input_s,[(s/1)*4,s*4,s*4])
    f[:,:,:,15] =  gaussian_filter(input_s,[(s/1)*4,s*4,s*4])
    fe = vigra.filters.structureTensorEigenvalues(input_ss.astype(np.float32),\
                                                  [(s/1)*4,s*4,s*4],[(s/1)*8,s*8,s*8])
    f[:,:,:,16] = fe[:,:,:,0]
    f[:,:,:,17] = fe[:,:,:,1]
    f[:,:,:,18] = fe[:,:,:,2]
    
    f[:,:,:,19] = gaussian_gradient_magnitude(input_s, s*8)
    f[:,:,:,20] = gaussian_laplace(input_s,s*8)
    f[:,:,:,21] = gaussian_filter(input_s,s*8)
    fe = vigra.filters.structureTensorEigenvalues(input_ss.astype(np.float32),\
                                                  [(s/1)*8,s*8,s*8],[(s/1)*16,s*16,s*16])
    f[:,:,:,22] = fe[:,:,:,0]
    f[:,:,:,23] = fe[:,:,:,1]
    f[:,:,:,24] = fe[:,:,:,2]
    
    out  = {}
    for z in range(0,imSizeZ):
        out[z] = f[z,:,:,:]
    return out

def update_training_samples_fn_ext(par_obj,model_num):
    """Collects the pixels which will be used for training sub-sampling as required."""
    region_size = 0
    for b in range(0,par_obj.saved_ROI.__len__()):
        rects = par_obj.saved_ROI[b]
        region_size += rects[4]*rects[3]        
    
    calc_ratio = par_obj.limit_ratio_size
    

    for b in range(0,par_obj.saved_ROI.__len__()):

        #Iterates through saved ROI.
        rects = par_obj.saved_ROI[b]
        img2load = rects[0]

        #Finds and extracts the features and output density for the specific regions.
        mImRegion = par_obj.feat_arr[rects[0]][rects[2]+1:rects[2]+rects[4],\
                                               rects[1]+1:rects[1]+rects[3],:]
        denseRegion = par_obj.dense_array[rects[0]][rects[2]+1:rects[2]+rects[4],\
                                                    rects[1]+1:rects[1]+rects[3]]
        #Find the linear form of the selected feature representation
        mimg_lin = np.reshape(mImRegion, (mImRegion.shape[0]*mImRegion.shape[1],mImRegion.shape[2]))
        #Find the linear form of the complementatory output region.
        dense_lin = np.reshape(denseRegion, (denseRegion.shape[0]*denseRegion.shape[1]))
        #Sample the input pixels sparsely or densely.
        if(par_obj.limit_sample == True):
            if(par_obj.limit_ratio == True):
                par_obj.limit_size = round(mImRegion.shape[0]*mImRegion.shape[1]/calc_ratio,0)

            #Randomly sample from input ROI or im a certain number (par_obj.limit_size) patches. With replacement.
            indices =  np.random.choice(int(mImRegion.shape[0]*mImRegion.shape[1]),\
                                        size=int(par_obj.limit_size), replace=True, p=None)
            #Add to feature vector and output vector.
            par_obj.f_matrix.extend(mimg_lin[indices])
            par_obj.o_patches.extend(dense_lin[indices]*4.0)
        else:
            #Add these to the end of the feature Matrix, input patches
            par_obj.f_matrix.extend(mimg_lin)
            #And the the output matrix, output patches
            par_obj.o_patches.extend(dense_lin)
        
        
    
def im_dimen_fn(par_obj,file_array):
    """Function which loads in Tiff stack or single png file to assess dimensions."""
    prevExt = [] 
    prevBitDepth=[] 
    prevNumCH =[]
    par_obj.numCH = 0
    par_obj.total_time_pt = 0
    for i in range(0,file_array.__len__()):
            n = str(i)
            imStr = str(file_array[i])
            par_obj.file_ext = imStr.split(".")[-1]
            if par_obj.file_ext == 'tif' or par_obj.file_ext == 'tiff':
                par_obj.tiff_file = tif_fn.TiffFile(imStr)
                meta = par_obj.tiff_file.series[0]
                order = meta.axes
                
                #Reads parameters from TIFF file input.
                for n,b in enumerate(order):
                    if b == 'T':
                        par_obj.total_time_pt = meta.shape[n]
                    if b == 'I':
                        par_obj.max_zslices = meta.shape[n]
                    if b == 'Z':
                        par_obj.max_zslices = meta.shape[n]
                    if b == 'Y':
                        par_obj.ori_height = meta.shape[n]
                    if b == 'X':
                        par_obj.ori_width = meta.shape[n]
                    if b == 'S':
                        par_obj.numCH = meta.shape[n]

                par_obj.bitDepth = meta.dtype
            else:
                 statusText = 'Status: Image format not-recognised. Please choose either png or TIFF files.'
                 return False, statusText

    statusText= str(file_array.__len__())+' Files Loaded.'
    return True, statusText
def im_pred_inline_fn(par_obj,ch_to_import):
    """Accesses TIFF file. Calculates features to indices present in par_obj.left_2_calc"""
    
    
    #Goes through the list of files.
    for b in par_obj.left_2_calc:
            #input string of file.
            imStr = str(par_obj.file_array[b])
            #Finds frames to be loaded for this file.
            frames = par_obj.frames_2_load[b]
            
            
            if par_obj.file_ext == 'tif' or par_obj.file_ext == 'tiff':
            #Make sure we have a tiff image, necessary for the stacks.
                if par_obj.features == '2D':
                    #For the case when we are calculating 2-D filters.
                    count = -1
                    for i in frames:
                        count = count+1
                        keyframe = (par_obj.max_zslices*par_obj.time_pt)+ i
                        if ch_to_import == 'default':
                            input_i = par_obj.tiff_file.asarray()[keyframe,:,:]
                        else:
                            input_i = par_obj.tiff_file.asarray()[keyframe,ch_to_import,:,:]
                        print 'Calculating Features for Image: ',(b+1),' Frame: ',(i+1)
                        feat = local_shape_features_2D(input_i.astype(np.float32),par_obj.feature_scale)
                        par_obj.feat_arr[count] = feat
                        par_obj.num_of_feat = feat.shape[2]
                elif par_obj.features == '3D':
                    #For the case when we are calculating 3-D filters.
                    if ch_to_import == 'default':
                        input_s = par_obj.tiff_file.asarray()[:,:,:]
                    else:
                        input_s = par_obj.tiff_file.asarray()[:,ch_to_import,:,:]
                    feat = local_shape_features_3D(input_s.astype(np.float32),par_obj.feature_scale)
                    print 'Calculating Features for Image: ',(b+1)
                    #export parameters and data
                    par_obj.num_of_feat = 25
                    par_obj.feat_arr = feat
                
    
    return
def process_imgs(par_obj,b,ch_to_import):
    """Imports ground-truth and input file. Calculates density represntation."""
    #We work on the full-size ground-truth file.
    GT_obj = tif_fn.TiffFile(par_obj.gt_file_array[b])
    #Convert to 3D array.
    GT_mov = GT_obj.asarray()

    ##### Create the HUMAN label data from Ground-truth data. #####
    #Generate empty container for the density label representation.
    par_obj.gt_sum[b] = np.sum(GT_mov)/255
    #Very important the size of the gaussian kernel
    sigma = par_obj.sigma;
    #The output density image.
    uncrop_dense_array = filters.gaussian_filter(GT_mov.astype(np.float32),\
                                                 sigma, order=0, output=None, mode='constant', cval=0.0)
    par_obj.dense_array = uncrop_dense_array[par_obj.frames_2_load[b][0]:par_obj.frames_2_load[b][-1]+1,:,:]
    #Make sure to add an image.
    par_obj.left_2_calc = [b]
    par_obj.gt_sum[b] = np.sum(GT_mov[par_obj.frames_2_load[b][0]:par_obj.frames_2_load[b][-1]+1,:,:])/255
    par_obj.gt_dense[b] = np.sum(par_obj.dense_array)/255

    ##### Calculates features for the given image.
    par_obj.tiff_file = tif_fn.TiffFile(par_obj.file_array[b])
    im_pred_inline_fn(par_obj,ch_to_import)
    

## Main Script

In [ ]:
class ParameterContainer():
    def __init__(self):
        self.file_array = []
        self.gt_file_array = []
        self.frames_2_load = {}
        self.feature_scale = 0.8
        self.p_size = 1
        self.feat_arr = {}
        self.features = '3D' #'2D' or '3D'
        self.limit_sample = True
        self.limit_ratio = True
        self.limit_ratio_size = 200
        self.max_depth = 20
        self.min_samples_split = 20 
        self.min_samples_leaf = 10  
        self.max_features = 7
        self.num_of_tree = 100
        self.resize_factor = 1
        self.gt_sum = {}
        self.gt_dense ={}
        self.f_matrix =[]
        self.o_patches=[]
        self.gt_sum = {}
        self.gt_dense ={}

tif_in_path = "data/" 
tif_out_path = "out/"
csv_out_Path = "out/"


for exp in [1]: #The experiment to be run.
    for num_of_train in [8]: #The number of training images used in case.
        for bc in range(0,10): #The number of times you want to repeat the experiment.
            
            #Where we store parameters for model and related.
            par_obj = ParameterContainer()
            

            if exp == 1:
                path = tif_in_path+'dataset1/'
                num_of_im = 30 #Number of images in the dataset.
                ss = ShuffleSplit(num_of_im, n_iter=1, test_size=15, train_size=num_of_train)
                par_obj.sigma =  [float(13),float(12),float(12)] #z, x, y
                ch_import_train = ch_import_test = 'default'
            if exp == 2:
                path = tif_in_path+'dataset2/'
                num_of_im = 30 #Number of images in the dataset.
                ss = ShuffleSplit(num_of_im, n_iter=1, test_size=15, train_size=num_of_train)
                par_obj.sigma =  [float(10),float(6),float(6)] #z, x, y
                ch_import_train = ch_import_test = 'default'
            if exp == 3:
                path = tif_in_path+'dataset3/'
                num_of_im = 26 #Number of images in the dataset.
                ss = ShuffleSplit(num_of_im, n_iter=1, test_size=15, train_size=num_of_train)
                par_obj.sigma =  [float(4),float(15),float(15)] #z, x, y
                ch_import_train = ch_import_test = 'default'
            if exp == 4:
                path = tif_in_path+'dataset4/'
                par_obj.sigma =  [float(1.8),float(1.8),float(1.8)] #z, x, y
                num_of_im = 10 #Number of images in the dataset.
                ss = ShuffleSplit(num_of_im, n_iter=1, test_size=2, train_size=num_of_train)
                ch_import_train = ch_import_test = 1
               


            for train_index, test_index in ss:
                 print("%s %s" % (train_index, test_index))
            
            #Load in data.
            for i in range(0,num_of_im):
                n = str(i).zfill(3)
                par_obj.file_array.append(path+'/image-final_0'+n+'.tif')
                par_obj.gt_file_array.append(path+'/image-dots_0'+n+'.tif')
                im_dimen_fn(par_obj,[par_obj.file_array[-1]])
                par_obj.frames_2_load[i] = np.arange(0,par_obj.max_zslices)
                par_obj.height = par_obj.ori_height
                par_obj.width = par_obj.ori_width
            
            print('Calculating features')
            for b in train_index:

                t1 = time.time()
                process_imgs(par_obj,b,ch_import_train)
                
                t2 = time.time()
                print "Image_id",b, 'time taken: ',t2-t1
                #Need to initialise are region for each image, which encompasses the whole plane.
                par_obj.saved_ROI = []
                for i in range(0,par_obj.frames_2_load[b].__len__()):
                    rects = (i,0,0,par_obj.width,par_obj.height)
                    par_obj.saved_ROI.append(rects)


                #Add the training data from this file.
                update_training_samples_fn_ext(par_obj,0)

            ##### Processes images by importing the image, the ground-truth image and calculates the features 

            par_obj.RF ={}
            par_obj.RF[0] = ExtraTreesRegressor(par_obj.num_of_tree, max_depth=par_obj.max_depth,\
                                                min_samples_split=par_obj.min_samples_split,\
                                                min_samples_leaf=par_obj.min_samples_leaf, \
                                                max_features=par_obj.max_features, bootstrap=True, n_jobs=-1)


            par_obj.pred_arr = {}
            par_obj.sum_pred = {}

            ######
            ###### Learns the ensemble of decision trees
            ######
            
            t3 = time.time()
            
            par_obj.RF[0].fit(par_obj.f_matrix, par_obj.o_patches)
            
            t4 = time.time()
            
            print 'Time to train',t4-t3
            
            ######
            ###### Evaluation of images.
            ######
            
            par_obj.final_prediction = {}
            time_taken_to_calc_feat = []
            time_taken_to_eval_trees = []
            
            print 'Calculating features for evaluation'
            par_obj.the_score = {}
            for b in test_index:
                par_obj.sum_pred ={}

                t1 = time.time()
                process_imgs(par_obj,b,ch_import_test)
                t2 = time.time()
                
                print 'Image_id: ',b,' time taken to calc features: ',t2-t1
        
                t1 = time.time()

                #Evaluates each frame using learnt framework.
                for it in range(0,par_obj.frames_2_load[b].__len__()):
                    evaluate_forest_3D(par_obj,0,count=it)
                    
                t2 = time.time()
                
                print 'Image_id: ',b,' time taken to eval trees: ',t2-t1
                
                
                par_obj.final_prediction[b] = 0
                for c in par_obj.sum_pred:
                    par_obj.final_prediction[b] += par_obj.sum_pred[c]
                
                
                #Outputs Image density image
                with tif_fn.TiffWriter(tif_out_path+'/exp_'+str(exp)+'_image-dense_0'+str(b)+'.tif',\
                                       bigtiff=True) as tif:
                    for i in range(par_obj.sum_pred.__len__()):
                        tif.save(par_obj.pred_arr[i], compress=0)
            
            
            
            ######
            ###### Outputs the performance.
            ######
            per_err = []
            for i,item in enumerate(test_index):
                per_err.append((1-(abs(par_obj.gt_dense[item] - \
                                       par_obj.final_prediction[item])/par_obj.gt_dense[item]))*100)
            print "output accuracy",np.average(per_err)
            